In [8]:
from pathlib import Path
from typing import Literal

from ipywidgets import interact
import matplotlib.pyplot as plt
import torch

In [15]:
PATH = Path("../fno_unet/runs/64x64_few/epoch-1.pt")
DATA = torch.load(PATH, map_location="cpu")

In [50]:
@interact(in_ch=(0, 512), out_ch=(0, 512), highlight_mode=["value", "sign"], name=[x for x in DATA.keys() if DATA[x].ndim == 4])  # pyright: ignore [reportUntypedFunctionDecorator]
def render(highlight_mode: Literal["value", "sign"], name: str, in_ch: int = 0, out_ch: int = 0) -> None:
    kernel = DATA[name][out_ch, in_ch]
    plt.figure()
    plt.subplot(111)
    if highlight_mode == "value":
        plt.imshow(kernel.numpy(), cmap="gray")
    elif highlight_mode == "sign":
        plt.imshow(kernel.sign().numpy(), cmap="gray")
    norm_kernel = (kernel - kernel.min()) / (kernel.max() - kernel.min())
    for y in range(kernel.shape[0]):
        for x in range(kernel.shape[1]):
            text_color = "white" if norm_kernel[y, x].item() <= 0.5 else "black"
            plt.text(x, y, f"{kernel[y, x]:.3f}", horizontalalignment="center", verticalalignment="center", color=text_color, fontsize=8)
    plt.gcf().canvas.draw()
    plt.show()
    print(f"1. Kernel-Moment: {kernel.mean().item():.3f}")
    print(f"2. Kernel-Moment: {kernel.var().item():.3f}")
    print()
    print("1. Layer-Moment: "+str(DATA[name].flatten(end_dim=1).mean(dim=0))[8:].replace(' '*8, ' '*17).translate(str.maketrans({'[': '', ']': '', ',': '', ')': ''})))
    print("2. Layer-Moment: "+str(DATA[name].flatten(end_dim=1).var(dim=0))[8:].replace(' '*8, ' '*17).translate(str.maketrans({'[': '', ']': '', ',': '', ')': ''})))

interactive(children=(Dropdown(description='highlight_mode', options=('value', 'sign'), value='value'), Dropdo…